In [1]:
import warnings
warnings.filterwarnings("ignore")
from utils import *
import pickle
OPBG = [ 'age at diagnosis (months)',            
            'ntbc dosis mg/kg/day',
            'ntbc levels (dbs)',
            'sca (urine)',
            'methionine dbs',
            'tyrosine dbs',
            'phenylalanine dbs',
            'methionine (plasma)',
            'tyrosine (plasma)',
            'phenylalanine (plasma)',
            'inr',
            'pt (sec)',
            'bili total',
            'gpt',
            'got',
            'ggt',
            'alkaline phosphatase',
            'alfa-fetoprotein']
    
df1 = pd.read_csv('data/tirosinemia.csv').loc[:,OPBG]
df2 = pd.read_csv('data/tirosinemia_italia.csv').loc[:,OPBG]
compare_dataframes(df1, df2)
def extract_transform(path, keep_cols, cols_rows_with_missing_values, new_binary_target):
    _ , spark_df = SparkDataProcessor().process_data(
        data_csv_path= path,
        )

    return prepare_spark_df(
        spark_df=spark_df,
        keep_cols=keep_cols,
        cols_rows_with_missing_values=cols_rows_with_missing_values,
        new_binary_target = new_binary_target,
    )

binary_target = 'Alpha-Fet'
df_train = extract_transform("data/tirosinemia.csv", OPBG,  ['alfa-fetoprotein', 'sca (urine)', 'ntbc levels (dbs)'],binary_target)
df_test  = extract_transform("data/tirosinemia_italia.csv", OPBG,  ['alfa-fetoprotein', 'sca (urine)', 'ntbc levels (dbs)'],binary_target)
ray_df_train = ray.put(df_train)
ray_df_test  = ray.put(df_test)

Both CSV files have the same number of columns: 18
Column names are the same.
Column order is the same.
All common columns have the same data types.
23/06/02 11:20:14 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 100.64.221.100 instead (on interface tailscale0)
23/06/02 11:20:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/02 11:20:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/06/02 11:20:16 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


2023-06-02 11:20:18,983	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


In [2]:
features : list[str] = df_train.columns.tolist()
features.remove(binary_target)


In [3]:
%%time

studies = make_multiple_studies(
    ray_df_train, 
    features = features,
    targets = [binary_target],
    n_trials=5,
    Independent_testset = True,
    Independent_testset_df = ray_df_test
)

studies = ray.get(studies)

(make_a_study pid=1616009) [I 2023-06-02 11:20:20,608] A new study created in memory with name: sca (urine)
(make_a_study pid=1616016) [I 2023-06-02 11:20:20,574] A new study created in memory with name: age at diagnosis (months)
(make_a_study pid=1616023) [I 2023-06-02 11:20:20,582] A new study created in memory with name: ntbc levels (dbs)
(make_a_study pid=1616022) [I 2023-06-02 11:20:20,584] A new study created in memory with name: ntbc dosis mg/kg/day
(make_a_study pid=1616023) [I 2023-06-02 11:20:32,410] Trial 1 finished with values: [0.79375, 0.12852682173252106] and parameters: {'seed': 4173, 'max_depth': 5, 'eta': 0.23602726195626492, 'subsample': 0.1369889282268515, 'lambda': 0.9151556944186781, 'alpha': 0.7088340148607216, 'scale_pos_weight': 3.1402084523238996, 'kfold_splits': 2}. 
(make_a_study pid=1616022) [I 2023-06-02 11:20:37,486] Trial 1 finished with values: [0.6125, 0.04656529426574707] and parameters: {'seed': 2673, 'max_depth': 10, 'eta': 0.3663096177150623, 'subs

CPU times: user 2.83 s, sys: 2.77 s, total: 5.59 s
Wall time: 2min 30s


In [5]:
with open('studies_independent_testset.pickle', 'wb') as handle:
    pickle.dump(studies, handle, protocol=pickle.HIGHEST_PROTOCOL)    